<a href="https://colab.research.google.com/github/Indian1999/szombatmi/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import json
import string
import random

import numpy as np
import nltk
from nltk import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [3]:
#Adatok beolvasása json-ből:
data_file = open("intents.json").read()
data = json.loads(data_file)
print(data)
print(type(data))
print(data["intents"])
print(data["intents"][0]["patterns"][1])

{'intents': [{'tag': 'hello', 'patterns': ['Hello', 'Hi there', 'Good morning', "What's up"], 'responses': ['Hey!', 'Hello', 'Hi!', 'Good morning!'], 'context': ''}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'job', 'patterns': ['What is your job', 'What is your work'], 'responses': ['My job is to make you feel like everything is okay.', 'I work to serve you as well as possible'], 'context': ''}, {'tag': 'age', 'patterns': ['What is your age', 'How old are you', 'When were you born'], 'responses': ['I was born in 2021'], 'context': ''}, {'tag': 'feeling', 'patterns': ['How are you today', 'How are you'], 'responses': ['I am feeling good, you?', 'Very good and you?', "Actually, I'm okay and you?"], 'context': ''}, {'tag': 'good', 'patterns': ['I am good too', 'I feel fine', 'Good !', 'Fine', 'I am good', 'I am great', 'great'], 'responses': ['That is perfect!', "So, ever

In [4]:
words   = [] # Szókészletet tárolja (Patterns)
classes = [] # tagek szókészlete
data_x  = [] # patternök
data_y  = [] # patternökhöz rendelt tagek

for intent in data["intents"]:
  for pattern in intent["patterns"]:
    tokens = nltk.word_tokenize(pattern)
    words.extend(tokens)
    data_x.append(pattern)
    data_y.append(intent["tag"])
  if intent["tag"] not in classes:
    classes.append(intent["tag"])

print(classes)
print(data_x)
print(data_y)
print(words)

['hello', 'noanswer', 'job', 'age', 'feeling', 'good', 'bad', 'actions', 'women', 'men', 'thanks', 'goodbye', 'city', 'action', 'wait', 'still there']
['Hello', 'Hi there', 'Good morning', "What's up", 'What is your job', 'What is your work', 'What is your age', 'How old are you', 'When were you born', 'How are you today', 'How are you', 'I am good too', 'I feel fine', 'Good !', 'Fine', 'I am good', 'I am great', 'great', 'I am feeling bad', 'No I am sad', 'No', 'What can you do', 'What can I ask you', 'Can you help me', 'Are you a girl', 'You are a women', 'Are you a men', 'Are you a boy', 'Thank you', 'Thank you very much', 'thanks', 'Goodbye', 'Good afternoon', 'Bye', 'Where do you live', 'What are you doing', 'Can you wait 2 minutes', 'Please wait', 'Wait 2 secs please', 'Are you still there?', 'Are you here?']
['hello', 'hello', 'hello', 'hello', 'job', 'job', 'age', 'age', 'age', 'feeling', 'feeling', 'good', 'good', 'good', 'good', 'good', 'good', 'good', 'bad', 'bad', 'bad', 'a

Szavak szótövesítése

In [5]:
lemmatizer = WordNetLemmatizer()
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words = sorted(set(words))
classes = sorted(set(classes))
print(words)
print(classes)

["'s", '2', 'a', 'afternoon', 'age', 'am', 'are', 'ask', 'bad', 'born', 'boy', 'bye', 'can', 'do', 'doing', 'feel', 'feeling', 'fine', 'girl', 'good', 'goodbye', 'great', 'hello', 'help', 'here', 'hi', 'how', 'i', 'is', 'job', 'live', 'me', 'men', 'minute', 'morning', 'much', 'no', 'old', 'please', 'sad', 'sec', 'still', 'thank', 'thanks', 'there', 'today', 'too', 'up', 'very', 'wait', 'were', 'what', 'when', 'where', 'woman', 'work', 'you', 'your']
['action', 'actions', 'age', 'bad', 'city', 'feeling', 'good', 'goodbye', 'hello', 'job', 'men', 'noanswer', 'still there', 'thanks', 'wait', 'women']


In [6]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [7]:
training = []
out_empty = [0] * len(classes)
print(out_empty)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
#print(data_x)
for index, doc in enumerate(data_x):
  bag_of_words = []
  text = lemmatizer.lemmatize(doc.lower())
  for word in words:
    if word in text:
      bag_of_words.append(1)
    else:
      bag_of_words.append(0)
  output_row = list(out_empty)
  output_row[classes.index(data_y[index])] = 1
  training.append([bag_of_words, output_row])


In [ ]:
random.shuffle(training)
training = np.array(training, dtype = object)
train_x = np.array(list(training[:,0]))
train_y = np.array(list(training[:,1]))
print(train_x)
print(train_y)

In [ ]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = "softmax"))
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer = opt, loss = "categorical_crossentropy", metrics=["accuracy"])
model.summary()

In [ ]:
model.fit(x=train_x, y = train_y, epochs = 150)

In [ ]:
def clean_text(text):
  tokens = nltk.word_tokenize(text)
  for word in tokens:
    tokens = [lemmatizer.lemmatize(word)]
  return tokens

def bag_of_words(text, vocab):
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens:
    for index, word in enumerate(vocab):
      if word == w:
        bow[index] = 1
  return np.array(bow)